### Solución Ejercicio 1.1: Listas ligadas ordenadas

En este ejercicio se pide implementar una lista ligada que esté ordenada crecientemente de acuerdo al valor de los nodos. Para esto, basta que al insertar un nuevo nodo en la lista ligada, esta lo inserte en la posición correspondiente según su valor. Al implentarlo, la cabeza tiene siempre el valor más pequeño y la cola el valor más grande. 

Deberás completar el método `agregar_ordenado` de la clase `ListaLigadaOrdenada`. Observa que no necesitamos el argumento `posicion` para agregar.

In [ ]:
class Nodo:
    def __init__(self, valor=None):
        self.valor = valor
        self.siguiente = None 


class ListaLigadaOrdenada:
    
    def __init__(self):
        self.cabeza = None
        self.cola = None
            
    def obtener(self, posicion):
        nodo_actual = self.cabeza
        for _ in range(posicion):
            if nodo_actual is not None:
                nodo_actual = nodo_actual.siguiente
        
        if nodo_actual is None:
            return None 
        return nodo_actual.valor
    
    
    def agregar(self, valor):
        
        nodo_nuevo = Nodo(valor)
        nodo_actual = self.cabeza
        
        if nodo_actual is None:
            self.cabeza = nodo_nuevo
            self.cola = self.cabeza
        
        else:
            if nodo_nuevo.valor < nodo_actual.valor:
                self.cabeza = nodo_nuevo
                nodo_nuevo.siguiente = nodo_actual
                return
            
            while nodo_nuevo.valor >= nodo_actual.valor:
                if nodo_actual.siguiente is not None:
                    if nodo_nuevo.valor < nodo_actual.siguiente.valor:
                        nodo_nuevo.siguiente = nodo_actual.siguiente
                        nodo_actual.siguiente = nodo_nuevo
                        return
                    else:
                        nodo_actual = nodo_actual.siguiente
                else:
                    # En este caso es mayor que todos
                    nodo_actual.siguiente = nodo_nuevo
                    self.cola = nodo_nuevo
                    return
                
    def __repr__(self):
        string = ""
        nodo_actual = self.cabeza
        while nodo_actual is not None:
            string = f"{string}{nodo_actual.valor} → "
            nodo_actual = nodo_actual.siguiente
        return string

In [ ]:
lista_ordenada = ListaLigadaOrdenada()
lista_ordenada.agregar(1)
print(lista_ordenada)
lista_ordenada.agregar(3)
print(lista_ordenada)
lista_ordenada.agregar(2)
print(lista_ordenada)
lista_ordenada.agregar(2.5)
print(lista_ordenada)
lista_ordenada.agregar(0)
print(lista_ordenada)

### Solución Ejercicio 2.3: Implementar una búsqueda BFS con profundidad máxima.

En este ejercicio debes implementar una forma de búsqueda que combina BFS con búsqueda en profundidad. Concretamente tienes que reimplementar el método `iter` para que ahora no sólo retorne nodos, si no que también la profundidad a las que se encuentra cada uno. Luego, tienes que implementar `dbfs_search` que recibe `max_depth` como argumento e imprime los nodos cuya profundidad es menor que el `max_depth`. Además, deben entregarse ordenados según profundidad.

In [ ]:
# textwrap tiene varias funciones convenientes para el manejo de strings
from textwrap import indent

class Arbol:
    """
    Esta clase representa un árbol
    
    La estructura es recursiva, de manera que cada nodo es la raíz de un sub-árbol.
    Los nodos hijos pueden ser guardados en una estructura, como lista o diccionario.
    En este ejemplo, los nodos hijos serán guardados en un diccionario.
    """

    def __init__(self, id_nodo, valor=None, padre=None):
        """
        Inicializa la estructura básica del árbol.
        
        Tiene un identificador propio, la referencia a su padre, el valor almacenado
        y una estructura con sus hijos.
        """
        self.id_nodo = id_nodo
        self.padre = padre
        self.valor = valor
        self.hijos = {}
        

    def obtener_nodo(self, id_nodo):
        """
        Obtiene el nodo con el id dado, en forma recursiva
        
        A partir del nodo actual, buscamos el nodo 'id_nodo' entre sus hijos
        y lo retornamos si existe.
        """
        # Caso base: ¡Lo encontramos!
        if self.id_nodo == id_nodo:
            return self

        # Buscamos recursivamente entre los hijos
        for hijo in self.hijos.values():
            nodo = hijo.obtener_nodo(id_nodo)
            # Si lo encontró, lo retornamos
            if nodo is not None:
                return nodo
        
        # Si no lo encuentra, retorna None
        return None


    def agregar_nodo(self, id_nodo, valor, id_padre):
        """Agrega un nodo con el id y valor dado, como hijo del nodo con el id 'id_padre'"""
        # Primero, tenemos que encontrar al padre
        padre = self.obtener_nodo(id_padre)
        # En caso de que el padre no exista no hacemos nada
        if padre is None:
            return
        
        # Creamos el nodo
        # Nos aseguramos de que el nodo nuevo sea del mismo tipo que la raíz
        # La siguiente línea es equivalente a Arbol(id_nodo, valor, padre) por ahora
        nodo = type(self)(id_nodo, valor, padre) # Esto lo ocuparemos cuando sea otra clase que herede de Arbol
        # Agregamos el nodo como hijo de su padre
        padre.hijos[id_nodo] = nodo
        
        
    def __repr__(self):
        """
        Entrega una representación del árbol, en forma recursiva.
        
        Para ello, tenemos que pedir la representación de cada hijo recursivamente. 
        Esto nos lleva a recorrer todos los nodos del árbol.
        """
        # Texto de este nodo.
        texto = f"id: {self.id_nodo}, valor: {self.valor}"
        # Si el nodo es hoja, se avisa de ello.
        # Si el nodo no es hoja, se deja un salto de línea para poder nombrar a los hijos.
        texto += ', nodo hoja' if len(self.hijos) == 0 else '\n'

        # Extrae el repr a cada hijo, en forma recursiva.
        repr_hijos = [repr(hijo) for hijo in self.hijos.values()]
        
        # Indentamos cada línea del texto de los hijos con tres espacios.
        # Esto es para que se note el nivel del nodo.
        texto_hijos = [indent(texto_hijo, "   ") for texto_hijo in repr_hijos]
        
        # Usamos join para juntar todos los strings anteriores con un salto de línea entre cada uno.
        return texto + "\n".join(texto_hijos)

In [ ]:
from collections import deque

class ArbolDBFS(Arbol):
    
    def __iter__(self):    
        # Utilizamos una cola para almacenar los nodos por visitar
        cola = deque()
        # El primer nodo a visitar será la raíz
        cola.append((self, 0))
        # Mientras queden nodos por visitar en la cola
        while len(cola) > 0:     
            # Extraemos el primero de la cola
            nodo_actual , actual_depth = cola.popleft() 
            # Entregamos el nodo actual que se recorre
            yield nodo_actual, actual_depth
            # Agregamos todos los nodos hijos a la cola y actualizamos la profundidad
            for hijo in nodo_actual.hijos.values():
                cola.append((hijo, actual_depth + 1))
    
    def dbfs_search(self, max_depth):
        for node, depth in self:
            if depth <= max_depth:
                print(f'Nodo con id {node.id_nodo} a profundidad {depth}')

In [ ]:
T = ArbolDBFS(0, 10, 2)
T.agregar_nodo(1, 8, 0)
T.agregar_nodo(3, 12, 0)
T.agregar_nodo(2, 9, 1)
T.agregar_nodo(4, 5, 3)
T.agregar_nodo(5, 14, 3)
T.agregar_nodo(6, 20, 3)
T.agregar_nodo(8, 4, 2)
T.agregar_nodo(7, 8, 4)
T.agregar_nodo(9, 15, 6)
T.agregar_nodo(10, 6, 6)
T.dbfs_search(2)